In [1]:
import numpy as np
from numba import jit
import time
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import math
from scipy.stats import norm
import random

## Part II: Estimation of Sensitivities in MC

In [2]:
def Delta_Analytical_Call(S, K, r, sigma, tau):
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    return norm.cdf(d1)

def Delta_Analytical_Put(S, K, r, sigma, tau):
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    return norm.cdf(d1) - math.e**-r

In [3]:
@jit(nopython=True, fastmath=True, parallel=True)
def European_Call(T, K, r, S, sigma, trials, z):
    '''
    This function calculates the value of an European Call option
    Arguments: maturity, strike price, interest rate, stock price, volaility, number of trials
    Returns: Array of size trial with values of european puts
    '''
    
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    payoff_array = np.zeros(trials)

    for i in range(trials):
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*z[i])
        
        if S_cur-K > 0:
            payoff_array[i] = (S_cur-K)*np.exp(-r*T)
        else:
            payoff_array[i] = 0

    return payoff_array


@jit(nopython=True, fastmath=True, parallel=True)
def Digital_Call(T, K, r, S, sigma, trials, z):
    
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    payoff_array = np.zeros(trials)
    
    for i in range(trials):
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*np.random.normal())
        
        if S_cur > K:
            payoff_array[i] = 1 * math.e**(-r*T)
        else:
            payoff_array[i] = 0

    return payoff_array

In [4]:
def get_delta_call(kwargs, S, e, seed=None):
    z=np.zeros(kwargs['trials'])
    for i in range(kwargs['trials']):
        z[i] = np.random.normal()
        
    kwargs['z'] = z   
    kwargs['S'] = S
    
    V = European_Call(**kwargs)
    
    if not seed:
        for i in range(kwargs['trials']):
            z[i] = random.normalvariate(0, 1)
    kwargs['S'] = S + e
    kwargs['z'] = z   
    
    Ve = European_Call(**kwargs)
    
    return (Ve-V)/ e


def get_delta_digital(kwargs, S, e, seed=None):
    z=np.zeros(kwargs['trials'])
    for i in range(kwargs['trials']):
        z[i] = np.random.normal()
        
    kwargs['z'] = z   
    kwargs['S'] = S
    
    V = Digital_Call(**kwargs)
    
    
    
    if not seed:
        for i in range(kwargs['trials']):
            z[i] = random.normalvariate(0, 1)
            
    kwargs['S'] = S + e
    kwargs['z'] = z   
    Ve = Digital_Call(**kwargs)
    
    return (Ve-V)/ e

In [5]:
analytical = Delta_Analytical_Call(100, 99, 0.06, 0.2, 1)
analytical

0.6737355117348961

In [6]:
kwargs = {}
kwargs['T'] = 1
kwargs['K'] = 99
kwargs['r'] = 0.06
kwargs['sigma'] = 0.2
kwargs['trials'] = 10**4

In [7]:
S = 100
e = .02

delta_matrix_noseeds = [get_delta_call(kwargs, S, e, None) for x in range(100)]
delta_matrix_seeds = [get_delta_call(kwargs, S, e, 100) for x in range(100)]

In [8]:
abs(analytical - np.mean(delta_matrix_seeds)) / analytical * 100

0.24623078305791687

In [9]:
abs(analytical - np.mean(delta_matrix_noseeds)) / analytical * 100

174.71012564317166

In [10]:
eps = [0.01,0.02,0.5]
size = [4,5,6,7]

In [11]:
seeded = np.matrix(np.zeros(12))
seeded.shape = (4,3)

unseeded = np.matrix(np.zeros(12))
unseeded.shape = (4,3)

In [12]:
for row in range(len(size)):
    for column in range(len(eps)):
        kwargs['trials'] = 10**size[row]
        e = eps[column]
        seeded[row,column] = abs(analytical - np.mean(get_delta_call(kwargs, S, e, 100))) / analytical * 100
        unseeded[row,column] = abs(analytical - np.mean(get_delta_call(kwargs, S, e,None))) / analytical * 100

In [13]:
unseeded

matrix([[ 730.23441658, 1269.06582439,    2.90958518],
        [ 543.02836525,  178.28747807,    4.87835408],
        [ 173.63007441,  278.56246521,    7.05173096],
        [  74.76633269,   20.6317627 ,    2.58210839]])

In [14]:
seeded

matrix([[1.09928057, 0.30578237, 0.17494403],
        [0.00810297, 0.0283348 , 0.3860533 ],
        [0.09224299, 0.05925032, 0.78264014],
        [0.0212434 , 0.0058684 , 0.6614007 ]])